下ごしらえ

In [2]:
import numpy as np
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from datetime import datetime
import time
import sys
# ライブラリまでのディレクトリ定義
sys.path.append('../ml-scratch/utils') 
#sys.path.append('../') # colaboratory用

import fc, get_mini_batch, relu, soft_max

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# 前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# 学習データをスプリット
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [4]:
print(X_train.shape)

(48000, 28, 28)


## 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成

In [89]:
import numpy as np

class SimpleConv1d:
    """
    チャンネル数を1に限定した1次元畳み込み層クラス
    Parameters
    ----------
    initializer : 初期化方法のインスタンス
    stride : int
      ストライド
    """
    
    def __init__(self, W, B, optimizer, stride=1):
        self.W = W
        self.B = B
        self.stride = stride
        self.optimizer = optimizer
        self.X = None
        self.A = None
        self.dW = np.zeros(len(W))
        self.dX = None
        
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        
        self.X = X.copy()                                  # backwardで使うから
        A = np.array([])                                   # 返却用
        F_idx = np.arange(len(self.W))          # フィルタのindex
        
        # フィルタ終点がX終点にたどり着くまでやる
        while F_idx[-1] <= len(self.X) - 1:
            A = np.append(A, X[F_idx] @ self.W + self.B)
            # フィルタをストライド分移動
            F_idx += self.stride
            
        return A
    
    
    def backward(self, y, A):
        """
        バックワード
        Parameters
        ----------
        y : 次の形のndarray, shape (batch_size, n_nodes2)
            正解値
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        
        self.dX = np.zeros(len(self.X))         # 初期化
        self.A = y - A                                       # パラメータ更新で使うためインスタンス変数化
        
        d_idx = np.arange(len(self.W))     # deltaX計算用index
        
        # deltaA分やる
        for s in range(len(self.A)):
            self.dX[d_idx] += self.A[s] * self.W
            d_idx += 1    # 右方向
                    
        # self.W self.Bの更新
        self = self.optimizer.update(self)
        
        return self.dX

In [90]:
import numpy as np

class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    
    def __init__(self, lr):
        self.lr = lr
        
        
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """
        
        i_s_idx = np.arange(len(layer.W))     # deltaX計算用index
        
        # deltaA分やる
        for s in range(len(layer.A)):
            layer.dW += layer.A[s] * layer.X[i_s_idx]
            i_s_idx += 1     # 右方向
            
        layer.W -= self.lr * layer.dW
        
        layer.dB = np.sum(layer.A, axis=0)
        layer.B -= self.lr * layer.dB
        
        return layer

## 【問題2】1次元畳み込み後の出力サイズの計算

In [91]:
def calc_output_size(n_in_size, pd_size, st_size, f_size):
    return int(1 + (n_in_size + (2 * pd_size) - f_size / st_size))

## 【問題3】小さな配列での1次元畳み込み層の実験

In [92]:
x = np.array([1,2,3,4]).astype(np.float)    # float型じゃないとブロードキャストできない
w = np.array([3, 5, 7]).astype(np.float)     # めんどくさいのでクラスで型変換は後回し
b = 1

sc1 = SimpleConv1d(w, b, SGD(0.1))
A = sc1.forward(x)
print('A : %s' % A)

# 誤差
y = np.array([45, 70])

dX = sc1.backward(y, A)
dW = sc1.dW
dB = sc1.dB
print('dW : %s' % dW)
print('dB : %s' % dB)
print('dX : %s' % dX)
print('outputsize : %s' % calc_output_size(len(sc1.X), 0, sc1.stride, len(sc1.W)))

A : [35. 50.]
dW : [ 50.  80. 110.]
dB : 30.0
dX : [ 30. 110. 170. 140.]
outputsize : 2


In [95]:
x = np.array([1,2,3,4,6]).astype(np.float)
w = np.array([3, 5, 7]).astype(np.float)
b = 1

sc1 = SimpleConv1d(w, b, SGD(0.1))
A = sc1.forward(x)
print('A : %s' % A)

# 誤差
y = np.array([45, 70, 30])

dX = sc1.backward(y, A)
dW = sc1.dW
dB = sc1.dB
print('dW : %s' % dW)
print('dB : %s' % dB)
print('dX : %s' % dX)
print('outputsize : %s' % calc_output_size(len(sc1.X), 0, sc1.stride, len(sc1.W)))

A : [35. 50. 72.]
dW : [ -76.  -88. -142.]
dB : -12.0
dX : [  30.  110.   44.  -70. -294.]
outputsize : 3


## 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成

In [ ]:
import numpy as np

class Conv1d:
    """
    チャンネル数を限定しない1次元畳み込み層クラス
    Parameters
    ----------
    initializer : 初期化方法のインスタンス
    stride : int
      ストライド
    """
    
    def __init__(self, W, B, optimizer, stride=1):
        self.W = W
        self.B = B
        self.stride = stride
        self.optimizer = optimizer
        self.sF = None
        self.X = None
        self.A = None
        self.dW = np.zeros(len(W))
        self.dX = None
        
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        
        self.X = X
        F = len(self.W)          # フィルタサイズ
        self.sF = F -1             # フィルタ-1
        A = np.empty(self.sF)     # 返却用
        
        for i in range(self.sF):
            i_s = F + i               # i + s
            A[i] = X[i : i_s] @ self.W + self.B
        
        return A
    
    
    def backward(self, y, A):
        """
        バックワード
        Parameters
        ----------
        y : 次の形のndarray, shape (batch_size, n_nodes2)
            正解値
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        # 初期化
        self.dX = np.zeros(len(self.X))
        
        # パラメータ更新で使うためインスタンス変数化
        self.A = y - A
        
        # self.W self.Bの更新
        self = self.optimizer.update(self)
              
         # TODO:めっちゃイケてないその２
        for s in range(len(self.A)):
            for i in range(len(self.X)-1):
                i_s = i + s
                self.dX[i_s] += self.A[s] * self.W[i]
        
        return self.dX

## 【問題5】学習・推定
## CNN分類器クラスの作成

In [ ]:
import numpy as np
import logging                                                                     # ログ
from datetime import datetime                                        # 時間のやつ
from sklearn.preprocessing import OneHotEncoder       # ワンホットのやつ
from tqdm import tqdm                                                     # 進捗バーを出してくれるやつ


class Scratch1dCNNClassifier():
    """
    ニューラルネットワーク分類器
    """

    def __init__(self, batch_size=10, n_epochs=20,  n_nodes=400, layer=3, verbose=True,
                            sigma=1e-2, lr=1e-2, activation=None, optimizer='sgd'):

        self.batch_size = batch_size     # バッチサイズ
        self.n_epochs = n_epochs         # エポック数 
        self.n_input = n_nodes              # 初回のノード数
        self.layer = layer                          # 層の数
        self.verbose = verbose               # 学習過程出力フラグ
        self.activation = activation        #活性化関数(文字列)
        self.FCs = []                                  # FCインスタンス格納用
        self.activations = []                     # 活性化関数インスタンス格納用
        self.loss_ = []                              # 学習用データの学習過程格納用
        self.loss_val_ = []                       # 検証用データの学習過程格納用
        
        # 初期化・最適化クラスインスタンス作成
        if activation == 'relu':
            self.initializer = HeInitializer()
        else:
            self.initializer = XavierInitializer()
        
        if optimizer == 'sgd':
            self.optimizer = SGD(lr)
        elif optimizer == 'adagrad':
            self.optimizer = AdaGrad(lr)
            
        # ワンホットライブラリのインスタンス作成
        self.enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
        
        # ログレベルを DEBUG に変更
        time_stamp = datetime.now().strftime('%Y%m%d')
        logging.basicConfig(filename='../tmp/sprint11_' + time_stamp + '.log', level=logging.DEBUG)
        
    
    def fit(self, X, y, X_val=None, y_val=None):
        """
        ニューラルネットワーク分類器を学習する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            学習用データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            学習用データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証用データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証用データの正解値
        """
        
        # ワンホット化
        y = self.enc.fit_transform(y[:, np.newaxis])
        
        # 検証用データがある場合
        if X_val is not None:
            y_val= self.enc.fit_transform(y_val[:, np.newaxis])


        # 学習用データから特徴量とクラス数を取得
        n_features = X.shape[1]
        n_output = y.shape[1]

        # 初期化
        n_nodes1 = self.n_input
        
        # 各層のFCインスタンス作成
        for i in range(self.layer):
            if i == (self.layer - 1):     # 出力層
                fc = FC(n_nodes2, n_output, self.initializer, self.optimizer)
                activation = Softmax()
            else:
                if i == 0:                       # 入力層
                    fc = FC(n_features, n_nodes1, self.initializer, self.optimizer)
                else:
                    n_nodes2 = int(n_nodes1 / 2)   # TODO: 多分よロしくない
                    fc = FC(n_nodes1, n_nodes2, self.initializer, self.optimizer)
                    n_nodes1 = n_nodes2
                
                # 出力層以外は指定された活性化関数をインスタンス化
                if self.activation == 'sigmoid':
                    activation = Sigmoid()
                elif self.activation == 'tanh':
                    activation = Tanh()
                elif self.activation == 'relu':
                    activation = ReLU()
                
            self.FCs.append(fc)                           # 各自格納
            self.activations.append(activation)


        # エポックごとに進捗率を計測
        for e in tqdm(range(self.n_epochs)):
            # ミニバッチ化
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch_size)
            # ロス格納用
            loss_ary = []
            
            # Xのn_samples / batch_size数分ループ処理
            for mini_X_train, mini_y_train in get_mini_batch:
                                            
                # forward propagation
                for i in range(self.layer):
                    if i == 0:                              # 入力層
                        A = self.FCs[i].forward(mini_X_train)
                        Z = self.activations[i].forward(A)
                    else:
                        A = self.FCs[i].forward(Z)
                        Z = self.activations[i].forward(A)
                
                # back propagation
                for i in range(self.layer):
                    n_FC = self.layer - i - 1      # インスタンス逆指定用
                    if i == 0:                               # 入力層
                        dA, loss = self.activations[n_FC].backward(Z, mini_y_train)
                        loss_ary.append(loss)
                    else:                                     # 出力層
                        dA = self.activations[n_FC].backward(dZ)
                        
                    dZ = self.FCs[n_FC].backward(dA)

                    
            #誤差を格納
            self.loss_.append(np.mean(loss_ary))
                        
            # 検証用データがある場合
            if X_val is not None:
                # forward propagation
                for i in range(self.layer):
                    if i == 0:                              # 入力層
                        A = self.FCs[i].forward(X_val)
                        Z = self.activations[i].forward(A)
                    else:
                        A = self.FCs[i].forward(Z)
                        Z = self.activations[i].forward(A)
                
                # 
                dA, loss_val = self.activations[self.layer-1].backward(Z, y_val)

                #誤差を格納
                self.loss_val_.append(np.mean(loss_val))
                            

            # フラグがTrueであればログ出力
            if self.verbose:
                logging.info('forward propagation %sエポック目 sum: %s shape: %s', e+1, np.sum(A), A.shape)
                logging.info('forward propagation %sエポック目 sum: %s shape: %s', e+1, np.sum(Z), Z.shape)
                logging.info('backward propagation %sエポック目 sum: %s shape: %s', e+1, np.sum(dA), dA.shape)
                logging.info('backward propagation %sエポック目 sum: %s shape: %s', e+1, np.sum(dZ), dZ.shape)
                logging.info('loss %sエポック目 : %s', e+1, np.sum(loss))

        return self


    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            学習データ

        Returns
        -------
        y_pred :  次の形のndarray, shape (n_samples, 1)
            推定結果
        """
        
        # forward propagation
        for i in range(self.layer):
            if i == 0:                              # 入力層
                A = self.FCs[i].forward(X)
                Z = self.activations[i].forward(A)
            else:
                A = self.FCs[i].forward(Z)
                Z = self.activations[i].forward(A)
        
        # 一番確率が高いラベルを予測値に
        y_pred = np.argmax(Z, axis=1)
        
        return y_pred
    